# Data Preparation  
Consisting of **Data Cleaning**, **Data Exploration**, and **Feature Engineering**.


## Data Discovery + Cleaning

In [91]:
# imports
import pandas as pd
import numpy as np
import scipy.stats

In [92]:
TRAIN_PATH = "cattle_data_train.csv"
TEST_PATH = "cattle_data_test.csv"

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)

Train Shape: (210000, 36)
Test Shape: (40000, 35)


In [93]:
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210000 entries, 0 to 209999
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Cattle_ID                210000 non-null  object 
 1   Breed                    210000 non-null  object 
 2   Climate_Zone             210000 non-null  object 
 3   Management_System        210000 non-null  object 
 4   Age_Months               210000 non-null  int64  
 5   Weight_kg                210000 non-null  float64
 6   Parity                   210000 non-null  int64  
 7   Lactation_Stage          210000 non-null  object 
 8   Days_in_Milk             210000 non-null  int64  
 9   Feed_Type                210000 non-null  object 
 10  Feed_Quantity_kg         199519 non-null  float64
 11  Feeding_Frequency        210000 non-null  int64  
 12  Water_Intake_L           210000 non-null  float64
 13  Walking_Distance_km      210000 non-null  float64
 14  Graz

,Cattle_ID,Breed,Climate_Zone,Management_System,Age_Months,Weight_kg,Parity,Lactation_Stage,Days_in_Milk,Feed_Type,...,BVD_Vaccine,Rabies_Vaccine,Previous_Week_Avg_Yield,Body_Condition_Score,Milking_Interval_hrs,Date,Farm_ID,Feed_Quantity_lb,Mastitis,Milk_Yield_L
0,CATTLE_133713,Holstein,Tropical,Intensive,114,544.8,4,Mid,62,Concentrates,...,0,1,6.31,3.0,12,2024-01-15,FARM_0301,36.8235,1,12.192634
1,CATTLE_027003,Holstein,Arid,Mixed,136,298.9,4,Mid,213,Crop_Residues,...,0,0,17.16,4.0,12,2023-10-31,FARM_0219,NaN,0,14.717031
2,CATTLE_122459,Holstein,Tropical,Semi_Intensive,64,336.6,4,Late,16,Hay,...,1,0,4.07,3.5,12,2024-05-20,FARM_0802,16.0965,0,14.006142
3,CATTLE_213419,Jersey,Mediterranean,Intensive,58,370.5,1,Early,339,Crop_Residues,...,0,0,10.23,3.0,24,2024-07-22,FARM_0034,40.7925,0,24.324325
4,CATTLE_106260,Guernsey,Subtropical,Intensive,84,641.5,6,Early,125,Mixed_Feed,...,1,1,20.68,3.0,12,2023-01-03,FARM_0695,33.7365,1,12.023074


In [94]:
print("Missing values in training data")
print(train.isna().sum().sort_values(ascending=False).head(20))
print()
print("Missing values in testing data")
print(test.isna().sum().sort_values(ascending=False).head(20))

Missing values in training data
Feed_Quantity_kg           10481
Feed_Quantity_lb           10481
Housing_Score               6279
Cattle_ID                      0
Brucellosis_Vaccine            0
HS_Vaccine                     0
BQ_Vaccine                     0
Anthrax_Vaccine                0
IBR_Vaccine                    0
BVD_Vaccine                    0
Rabies_Vaccine                 0
Previous_Week_Avg_Yield        0
Body_Condition_Score           0
Milking_Interval_hrs           0
Date                           0
Farm_ID                        0
Mastitis                       0
FMD_Vaccine                    0
Humidity_percent               0
Breed                          0
dtype: int64

Missing values in testing data
Feed_Quantity_kg           2015
Feed_Quantity_lb           2015
Housing_Score              1221
FMD_Vaccine                   0
Brucellosis_Vaccine           0
HS_Vaccine                    0
BQ_Vaccine                    0
Anthrax_Vaccine               0
IBR_Vac

While exploring the different categories the data had, we discovered that the data had both Feed_Quantity_kg and Feed_Quantity_lb. We suspected that this data was the same thing, but just in different rates and wanted to check. We quickly checked if they were missing the same data and we discovered that when one is missing, so is the other. Therefore, they are the same and we only need one so we dropped Feed_Quantity_lb. Additionally, Cattle_ID isn't a useful field as it is unique to each cow so we can safely drop it.

In [95]:
(train["Feed_Quantity_kg"].isna() == train["Feed_Quantity_lb"].isna()).all()
mismatch_count = (train["Feed_Quantity_kg"].isna() != train["Feed_Quantity_lb"].isna()).sum()
print(mismatch_count)
train[train["Feed_Quantity_kg"].isna() != train["Feed_Quantity_lb"].isna()].head()
kg_na = train["Feed_Quantity_kg"].isna()
lb_na = train["Feed_Quantity_lb"].isna()

print("Are missing masks identical?:", (kg_na == lb_na).all())
print("Number of mismatches:", (kg_na != lb_na).sum())

train = train.drop("Feed_Quantity_lb", axis=1)
train = train.drop("Cattle_ID", axis=1)

0
Are missing masks identical?: True
Number of mismatches: 0


Next, we wanted to see if there were any negative values in any of the categories we suspected should never be negative, like things related to time (i.e. months, hours), weight (i.e. liters, kilograms, etc.), etc.

In [96]:
non_negatives = {"Age_Months": 0, "Weight_kg": 0, "Parity": 0, "Days_in_Milk": 0, "Feed_Quantity_kg": 0, "Water_Intake_L": 0, "Walking_Distance_km": 0, "Grazing_Duration_hrs": 0, "Rumination_Time_hrs": 0, "Resting_Hours": 0, "Humidity_percent": 0, "Previous_Week_Avg_Yield": 0, "Milking_Interval_hrs": 0, "Milk_Yield_L": 0}
for category in non_negatives:
    item = train[category]
    for i in item:
        if i < 0:
            non_negatives[category] += 1
for x in non_negatives:
    if non_negatives[x] != 0:
        print(f"{x}: {non_negatives[x]}")

Rumination_Time_hrs: 115627
Milk_Yield_L: 74


Oddly enough, it seems that Rumination_Time_hrs (Hours spent chewing cud per day), Milk_Yield_L (The total volume of milk produced by the cow in liters during the recorded milking period; this is the target variable to be predicted.) seem to be negative a lot.    

Therefore, we are going to drop the Rumination_Time_hrs column and the negative Milk_Yield_L rows.

In [97]:
if "Rumination_Time_hrs" in train.columns:
    train = train.drop(columns=["Rumination_Time_hrs"])
    print("Dropped Rumination_Time_hrs column.")
neg_rows = train[train["Milk_Yield_L"] < 0].shape[0]
print(f"Negative Milk_Yield_L rows removed: {neg_rows}")

train = train[train["Milk_Yield_L"] >= 0].reset_index(drop=True)
train.head()

Dropped Rumination_Time_hrs column.
Negative Milk_Yield_L rows removed: 74


,Breed,Climate_Zone,Management_System,Age_Months,Weight_kg,Parity,Lactation_Stage,Days_in_Milk,Feed_Type,Feed_Quantity_kg,...,IBR_Vaccine,BVD_Vaccine,Rabies_Vaccine,Previous_Week_Avg_Yield,Body_Condition_Score,Milking_Interval_hrs,Date,Farm_ID,Mastitis,Milk_Yield_L
0,Holstein,Tropical,Intensive,114,544.8,4,Mid,62,Concentrates,16.363455,...,1,0,1,6.31,3.0,12,2024-01-15,FARM_0301,1,12.192634
1,Holstein,Arid,Mixed,136,298.9,4,Mid,213,Crop_Residues,NaN,...,0,0,0,17.16,4.0,12,2023-10-31,FARM_0219,0,14.717031
2,Holstein,Tropical,Semi_Intensive,64,336.6,4,Late,16,Hay,7.198607,...,1,1,0,4.07,3.5,12,2024-05-20,FARM_0802,0,14.006142
3,Jersey,Mediterranean,Intensive,58,370.5,1,Early,339,Crop_Residues,18.694344,...,0,0,0,10.23,3.0,24,2024-07-22,FARM_0034,0,24.324325
4,Guernsey,Subtropical,Intensive,84,641.5,6,Early,125,Mixed_Feed,14.779198,...,0,1,1,20.68,3.0,12,2023-01-03,FARM_0695,1,12.023074


We then did some exploration regarding the Qualitative features.

In [98]:
qualitative_cols = train.select_dtypes(include=["object", "category"]).columns.tolist()
print("Qualitative features:", qualitative_cols)
for col in qualitative_cols:
    print(f"{col}: {train[col].nunique()} unique values")


Qualitative features: ['Breed', 'Climate_Zone', 'Management_System', 'Lactation_Stage', 'Feed_Type', 'Date', 'Farm_ID']
Breed: 7 unique values
Climate_Zone: 6 unique values
Management_System: 5 unique values
Lactation_Stage: 3 unique values
Feed_Type: 8 unique values
Date: 1095 unique values
Farm_ID: 1000 unique values


We now want to deal with the missing Housing Scores. We are assuming that Housing Score is somewhat correlated with the Farm_ID. A low variance score means we can say that they are properly correlated. This means that when a Housing Score is missing, we can just take the average of the Housing Score of that Farm_ID and plug it in. However, we also want to make sure that the Farm_ID isn't the only thing we're using to group this by, so we will look for the next best thing that lowers the variance when correlated.

In [99]:
df_hs = train.dropna(subset=["Housing_Score"])
farm_variances = df_hs.groupby("Farm_ID")["Housing_Score"].var()
farm_variances_sorted = farm_variances.sort_values(ascending=False)
farm_variances_sorted.head(10)   # top 10 highest variance

farm_means = df_hs.groupby("Farm_ID")["Housing_Score"].mean()
global_mean = train["Housing_Score"].mean()
missing_before = train["Housing_Score"].isna().sum()

qualitative_cols = train.select_dtypes(include=["object", "category"]).columns.tolist()
qualitative_cols = [c for c in qualitative_cols if c not in ["Farm_ID"]]

subgroup_results = {}

for col in qualitative_cols:
    subgroup_var = (
        df_hs.groupby(["Farm_ID", col])["Housing_Score"].var()
    )
    mean_var = subgroup_var.mean()
    
    subgroup_results[col] = mean_var
    print(f"{col}: mean within-group variance = {mean_var:.4f}")

sorted_subgroups = sorted(subgroup_results.items(), key=lambda x: x[1])
print("\n=== Best subgroup features after Farm_ID (lowest variance) ===")
for feat, val in sorted_subgroups:
    print(f"{feat}: {val:.4f}")

best_feature = sorted_subgroups[0][0]
print(f"\nBest subgroup after Farm_ID: {best_feature}")

Breed: mean within-group variance = 0.0409
Climate_Zone: mean within-group variance = 0.0411
Management_System: mean within-group variance = 0.0410
Lactation_Stage: mean within-group variance = 0.0410
Feed_Type: mean within-group variance = 0.0410
Date: mean within-group variance = 0.0407

=== Best subgroup features after Farm_ID (lowest variance) ===
Date: 0.0407
Breed: 0.0409
Management_System: 0.0410
Feed_Type: 0.0410
Lactation_Stage: 0.0410
Climate_Zone: 0.0411

Best subgroup after Farm_ID: Date


We get that Date and Breed have the lowest variance, so we will try group by Farm_ID then Date then Breed.

In [100]:
train["Date"] = pd.to_datetime(train["Date"], errors="coerce")
df_hs = train.dropna(subset=["Housing_Score"])
triple_group_var = (
    df_hs.groupby(["Farm_ID"])["Housing_Score"].var()
          .sort_values(ascending=False)
)
print("Top 10 highest variances (Farm_ID, Date):")
print(triple_group_var.head(10))

print("\nMean variance across groups:", triple_group_var.mean())

Top 10 highest variances (Farm_ID, Date):
Farm_ID
FARM_0102    0.048945
FARM_0877    0.048765
FARM_0532    0.048644
FARM_0645    0.048580
FARM_0795    0.048477
FARM_0015    0.048030
FARM_0989    0.047777
FARM_0411    0.047756
FARM_0212    0.047521
FARM_0150    0.047442
Name: Housing_Score, dtype: float64

Mean variance across groups: 0.041010327599322195


In [101]:
# We are going to use Farm_ID to get the means to replace the empty Housing Score columns.
train["Date"] = pd.to_datetime(train["Date"], errors="coerce")
df_hs = train.dropna(subset=["Housing_Score"])
farm_date_means = (
    df_hs.groupby(["Farm_ID"])["Housing_Score"]
         .mean()
)
missing_mask = train["Housing_Score"].isna()
keys = list(zip(train.loc[missing_mask, "Farm_ID"], train.loc[missing_mask, "Date"]))
train.loc[missing_mask, "Housing_Score"] = [
    farm_date_means.get(key, np.nan) for key in keys
]
farm_means = df_hs.groupby("Farm_ID")["Housing_Score"].mean()
still_missing = train["Housing_Score"].isna()

train.loc[still_missing, "Housing_Score"] = train.loc[still_missing, "Farm_ID"].map(farm_means)

global_mean = df_hs["Housing_Score"].mean()
train["Housing_Score"] = train["Housing_Score"].fillna(global_mean)

Now, we do the same thing for Feed_Quantity_kg.

In [102]:
df_fq = train.dropna(subset=["Feed_Quantity_kg"])
qualitative_cols = train.select_dtypes(include=["object", "category"]).columns.tolist()

subgroup_results_fq = {}
for col in qualitative_cols:
    subgroup_var = df_fq.groupby(col)["Feed_Quantity_kg"].var()
    mean_var = subgroup_var.mean()
    
    subgroup_results_fq[col] = mean_var
    print(f"{col}: mean within-group variance = {mean_var:.4f}")
sorted_subgroups_fq = sorted(subgroup_results_fq.items(), key=lambda x: x[1])

print("\n=== Best subgroup features (lowest mean variance) ===")
for feat, val in sorted_subgroups_fq:
    print(f"{feat}: {val:.4f}")
best_feature_fq = sorted_subgroups_fq[0][0]
print(f"\nBest single subgroup for Feed_Quantity_kg: {best_feature_fq}")

Breed: mean within-group variance = 14.6411
Climate_Zone: mean within-group variance = 15.7531
Management_System: mean within-group variance = 15.7533
Lactation_Stage: mean within-group variance = 15.7435
Feed_Type: mean within-group variance = 15.7533
Farm_ID: mean within-group variance = 15.7467

=== Best subgroup features (lowest mean variance) ===
Breed: 14.6411
Lactation_Stage: 15.7435
Farm_ID: 15.7467
Climate_Zone: 15.7531
Management_System: 15.7533
Feed_Type: 15.7533

Best single subgroup for Feed_Quantity_kg: Breed


We see that Breed has the lowest variance so we will see if we can group on Breed and then what is best next:

In [103]:
qualitative_cols = train.select_dtypes(include=["object", "category"]).columns.tolist()
qualitative_cols = [c for c in qualitative_cols if c not in ["Breed"]]

second_level_results = {}
for col in qualitative_cols:
    subgroup_var = df_fq.groupby(["Breed", col])["Feed_Quantity_kg"].var()
    mean_var = subgroup_var.mean()
    
    second_level_results[col] = mean_var
    print(f"Breed + {col}: mean variance = {mean_var:.4f}")

# Sort results from best (lowest variance) to worst
sorted_second_level = sorted(second_level_results.items(), key=lambda x: x[1])

print("\n=== Best second-level subgroup after Breed ===")
for feat, val in sorted_second_level:
    print(f"{feat}: {val:.4f}")

best_second_feature = sorted_second_level[0][0]
print(f"\nBest subgroup after Breed: {best_second_feature}")

Breed + Climate_Zone: mean variance = 14.6153
Breed + Management_System: mean variance = 14.7664
Breed + Lactation_Stage: mean variance = 14.6518
Breed + Feed_Type: mean variance = 14.8448
Breed + Farm_ID: mean variance = 15.8105

=== Best second-level subgroup after Breed ===
Climate_Zone: 14.6153
Lactation_Stage: 14.6518
Management_System: 14.7664
Feed_Type: 14.8448
Farm_ID: 15.8105

Best subgroup after Breed: Climate_Zone


Since grouping by Breed and any other item doesn't seem to improve the variance that much, we will just be using the mean after grouping by Breed to fill in the values for Feed_Quantity_kg.

In [104]:
df_fq = train.dropna(subset=["Feed_Quantity_kg"])
breed_means = df_fq.groupby("Breed")["Feed_Quantity_kg"].mean()
global_mean_fq = df_fq["Feed_Quantity_kg"].mean()
missing_before = train["Feed_Quantity_kg"].isna().sum()
train["Feed_Quantity_kg"] = train.apply(
    lambda row:
        breed_means[row["Breed"]]
        if pd.isna(row["Feed_Quantity_kg"]) and row["Breed"] in breed_means
        else (global_mean_fq if pd.isna(row["Feed_Quantity_kg"]) else row["Feed_Quantity_kg"]),
    axis=1
)

missing_after = train["Feed_Quantity_kg"].isna().sum()
print(f"Missing Feed_Quantity_kg before imputation: {missing_before}")
print(f"Missing Feed_Quantity_kg after imputation:  {missing_after}")
print("Feed_Quantity_kg imputation by Breed complete.")

Missing Feed_Quantity_kg before imputation: 10480
Missing Feed_Quantity_kg after imputation:  0
Feed_Quantity_kg imputation by Breed complete.


In [105]:
# SANITY CHECK: should produce all 0s
print("Missing values in training data")
print(train.isna().sum().sort_values(ascending=False).head(20))

Missing values in training data
Breed                      0
Housing_Score              0
Mastitis                   0
Farm_ID                    0
Date                       0
Milking_Interval_hrs       0
Body_Condition_Score       0
Previous_Week_Avg_Yield    0
Rabies_Vaccine             0
BVD_Vaccine                0
IBR_Vaccine                0
Anthrax_Vaccine            0
BQ_Vaccine                 0
HS_Vaccine                 0
Brucellosis_Vaccine        0
FMD_Vaccine                0
Humidity_percent           0
Climate_Zone               0
Ambient_Temperature_C      0
Resting_Hours              0
dtype: int64


We were a little confused about why Climate Zone and Farm ID weren't matched for how well it can be used to predict the Feed_Quantity_kg earlier so we decided to calculate all pairwise correlations. We thought that Climate Zone and Farm ID should have a correlatoin one of 1 because we believed thought that each Farm ID represented a singular farm and we thought that a farm could only be physically located in ONE climate zone.

In [26]:
# Helper functions that will be used to calculate correlation
from scipy.stats import chi2_contingency

def cramers_v(x, y):
    """Cramer's V for categorical-categorical"""
    confusion = pd.crosstab(x, y)
    if confusion.shape[0] == 1 or confusion.shape[1] == 1:
        return np.nan
    chi2 = chi2_contingency(confusion)[0]
    n = confusion.sum().sum()
    phi2 = chi2 / n
    r, k = confusion.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

def correlation_ratio(categories, values):
    """Eta: numeric-categorical correlation"""
    if categories.nunique() < 2:
        return np.nan
    mask = ~(categories.isna() | values.isna())
    categories = categories[mask]
    values = values[mask]
    overall_mean = values.mean()
    cat_means = values.groupby(categories).mean()
    cat_counts = categories.value_counts()
    numerator = ((cat_means - overall_mean)**2 * cat_counts).sum()
    denominator = ((values - overall_mean)**2).sum()
    if denominator == 0:
        return np.nan
    return np.sqrt(numerator / denominator)

In [27]:
# Build full correlation matrix
# WARNING: THIS TAKES A LONG TIME TO RUN AND YOU PROBABLY DON'T NEED TO RUN THIS AGAIN

df = train.copy()

for col in df.select_dtypes(include=["datetime", "datetimetz"]).columns:
    df[col] = df[col].astype(str)
for col in df.select_dtypes(include=["object", "category"]).columns:
    df[col] = df[col].astype(str)

cat_cols = df.select_dtypes(include=["object", "category"]).columns
num_cols = df.select_dtypes(include=["number"]).columns

cols = df.columns
corr_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)

for col1 in cols:
    for col2 in cols:
        if col1 == col2:
            corr_matrix.loc[col1, col2] = 1.0
            continue
        
        x = df[col1]
        y = df[col2]

        if col1 in num_cols and col2 in num_cols:
            corr = x.corr(y)
        elif col1 in cat_cols and col2 in cat_cols:
            corr = cramers_v(x, y)
        elif col1 in cat_cols and col2 in num_cols:
            corr = correlation_ratio(x, y)
        elif col1 in num_cols and col2 in cat_cols:
            corr = correlation_ratio(y, x)

        corr_matrix.loc[col1, col2] = corr

print("Full mixed-type correlation matrix computed (no errors!).")
corr_matrix

Full mixed-type correlation matrix computed (no errors!).


,Cattle_ID,Breed,Climate_Zone,Management_System,Age_Months,Weight_kg,Parity,Lactation_Stage,Days_in_Milk,Feed_Type,...,IBR_Vaccine,BVD_Vaccine,Rabies_Vaccine,Previous_Week_Avg_Yield,Body_Condition_Score,Milking_Interval_hrs,Date,Farm_ID,Mastitis,Milk_Yield_L
Cattle_ID,1.000000,0.009622,0.012727,0.012727,1.000000,1.000000,1.000000,0.012727,1.000000,0.012727,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.072196,0.068990,0.999894,0.999960
Breed,0.009622,1.000000,0.000000,0.000000,0.005589,0.003849,0.004786,0.002949,0.005921,0.000000,...,0.003969,0.006800,0.003847,0.005944,0.006236,0.006894,0.000000,0.007929,0.005234,0.003810
Climate_Zone,0.012727,0.000000,1.000000,0.003591,0.004803,0.003809,0.004739,0.003945,0.002720,0.000000,...,0.003718,0.006823,0.005482,0.004585,0.005295,0.002849,0.006596,0.000000,0.001843,0.002491
Management_System,0.012727,0.000000,0.003591,1.000000,0.005511,0.004266,0.006323,0.000000,0.004910,0.003906,...,0.003202,0.007188,0.006392,0.002875,0.005192,0.004873,0.000000,0.006986,0.005767,0.002515
Age_Months,1.000000,0.005589,0.004803,0.005511,1.000000,0.002440,0.000818,0.003735,0.001531,0.003566,...,-0.001736,-0.000058,-0.001715,-0.016075,-0.003157,-0.000456,0.071531,0.066575,-0.001243,-0.308830
Weight_kg,1.000000,0.003849,0.003809,0.004266,0.002440,1.000000,0.001750,0.002796,0.002686,0.006257,...,0.000292,-0.001241,0.000105,-0.001182,-0.000183,0.000695,0.072400,0.066643,0.001364,0.300240
Parity,1.000000,0.004786,0.004739,0.006323,0.000818,0.001750,1.000000,0.001655,-0.002350,0.003993,...,0.000565,-0.001753,-0.002392,0.001531,0.003643,0.001101,0.072680,0.070763,0.003858,-0.236244
Lactation_Stage,0.012727,0.002949,0.003945,0.000000,0.003735,0.002796,0.001655,1.000000,0.001216,0.000000,...,0.001042,0.001763,0.004157,0.178188,0.001897,0.001591,0.000000,0.011754,0.002295,0.090927
Days_in_Milk,1.000000,0.005921,0.002720,0.004910,0.001531,0.002686,-0.002350,0.001216,1.000000,0.008241,...,-0.001520,-0.002916,0.001641,-0.121992,-0.003565,-0.001565,0.073135,0.068093,-0.000035,-0.061054
Feed_Type,0.012727,0.000000,0.000000,0.003906,0.003566,0.006257,0.003993,0.000000,0.008241,1.000000,...,0.005526,0.004793,0.006639,0.002424,0.007457,0.005571,0.007539,0.006760,0.003202,0.005667


In [ ]:
# Save the correlation matrix so will never have to run that correlation matrix again
corr_pairs = (
    corr_matrix.stack()
    .reset_index()
    .rename(columns={"level_0": "Feature1", "level_1": "Feature2", 0: "Correlation"})
)

corr_pairs = corr_pairs[corr_pairs["Feature1"] < corr_pairs["Feature2"]]
corr_pairs = corr_pairs.iloc[corr_pairs["Correlation"].abs().argsort()[::-1]]
corr_pairs.to_csv("correlation_pairs_sorted.csv", index=False)
print("Saved sorted pairwise correlations to correlation_pairs_sorted.csv")

""" 
After getting all the pairwise correlations, we've seen that the maximum correlation
between two pairs is around 30%, so nothing too much of note we've determined.
"""

Saved sorted pairwise correlations to correlation_pairs_sorted.csv


" \nAfter getting all the pairwise correlations, we've seen that the maximum correlation\nbetween two pairs is around 30%, so nothing too much of note we've determined.\n"

We are a little confused as to why Farm_ID and Climate_Zone have 0 correlation. After a little more data exploration, we found out that each Farm_ID roughly has an equal number of each Climate_Zone, which is a little confusing since to us, logically, one Farm_ID represents one farm in one location and therefore should be in one Climate_Zone. In addition, we have enough other temperature related features that we think it is alright to drop this one. Therefore, we are dropping the Climate_Zone column completely. 

From the correlation pairings, this is the only pairing with a 0 correlation score that doesn't really make sense, so on the basis of correlation pairing, that's the only feature we will drop.


In [106]:
climate_counts = pd.crosstab(train['Farm_ID'], train['Climate_Zone'])

climate_counts.to_csv("farm_climate_zone_counts.csv")

print("Saved as farm_climate_zone_counts.csv")
print(climate_counts.head())

train = train.drop("Climate_Zone", axis=1)
train.head()

Saved as farm_climate_zone_counts.csv
Climate_Zone  Arid  Continental  Mediterranean  Subtropical  Temperate  \
Farm_ID                                                                  
FARM_0001       27           37             30           36         27   
FARM_0002       35           34             28           35         40   
FARM_0003       29           35             36           37         40   
FARM_0004       31           41             34           38         44   
FARM_0005       35           34             45           24         33   

Climate_Zone  Tropical  
Farm_ID                 
FARM_0001           42  
FARM_0002           28  
FARM_0003           27  
FARM_0004           40  
FARM_0005           42  


,Breed,Management_System,Age_Months,Weight_kg,Parity,Lactation_Stage,Days_in_Milk,Feed_Type,Feed_Quantity_kg,Feeding_Frequency,...,IBR_Vaccine,BVD_Vaccine,Rabies_Vaccine,Previous_Week_Avg_Yield,Body_Condition_Score,Milking_Interval_hrs,Date,Farm_ID,Mastitis,Milk_Yield_L
0,Holstein,Intensive,114,544.8,4,Mid,62,Concentrates,16.363455,2,...,1,0,1,6.31,3.0,12,2024-01-15,FARM_0301,1,12.192634
1,Holstein,Mixed,136,298.9,4,Mid,213,Crop_Residues,12.014374,4,...,0,0,0,17.16,4.0,12,2023-10-31,FARM_0219,0,14.717031
2,Holstein,Semi_Intensive,64,336.6,4,Late,16,Hay,7.198607,2,...,1,1,0,4.07,3.5,12,2024-05-20,FARM_0802,0,14.006142
3,Jersey,Intensive,58,370.5,1,Early,339,Crop_Residues,18.694344,2,...,0,0,0,10.23,3.0,24,2024-07-22,FARM_0034,0,24.324325
4,Guernsey,Intensive,84,641.5,6,Early,125,Mixed_Feed,14.779198,4,...,0,1,1,20.68,3.0,12,2023-01-03,FARM_0695,1,12.023074


Now we want to look for outliers in our code so we will describe every numerical feature.   

Result: From this, we manually looked through and determined most of the min and max's were pretty reasonable so we will keep this.


In [107]:
numerical_cols = train.select_dtypes(include=["number"]).columns
full_desc = train[numerical_cols].describe()
num_summary = full_desc.loc[["mean", "std", "min", "max"]].T
print(num_summary)

num_summary.to_csv("numerical_feature_summary.csv")
print("Saved to numerical_feature_summary.csv")

                               mean         std         min         max
Age_Months                83.473243   34.647286   24.000000  143.000000
Weight_kg                499.968245  144.651856  250.000000  750.000000
Parity                     3.499967    1.707315    1.000000    6.000000
Days_in_Milk             182.059226  105.029984    1.000000  364.000000
Feed_Quantity_kg          12.015898    3.868669    2.370284   25.454207
Feeding_Frequency          2.999081    1.413191    1.000000    5.000000
Water_Intake_L            80.037438   14.987507   14.207737  149.960210
Walking_Distance_km        4.034781    1.928525    0.500000   12.000000
Grazing_Duration_hrs       6.056783    2.867601    1.000000   14.000000
Resting_Hours             10.067635    2.865592    5.000000   18.000000
Ambient_Temperature_C     21.883828   11.695378  -11.863353   47.192325
Humidity_percent          59.813006   19.489647   10.000000  100.000000
Housing_Score              0.649079    0.199482    0.269460    1

## Feature Engineering

Now time to make the categorical features a little more efficient by converting them.   

One Hot Encode: 
- Breed
- Management_System 

Change to Ranges:
- Lactation_Stage (Early, Mid, Late)    

Farm_ID: will be converted to the ranking of how much milk the farm produces.
i.e. if FARM_0301 has a cow that produces 30L and 24.2L and is the most, that
FARM_ID will be 1.  

Date: convert to ordinal days from a base date of 2000-01-01.


In [108]:
# 1. One-hot encode selected columns
onehot_cols = ["Breed", "Management_System", "Feed_Type"]
train = pd.get_dummies(train, columns=onehot_cols, prefix=onehot_cols, drop_first=True)

# 2. Numeric encoding for Lactation_Stage
stage_map = {"Early": 1, "Mid": 2, "Late": 3}
train["Lactation_Stage"] = train["Lactation_Stage"].map(stage_map).astype(int)

# 3. Farm_ID → rank based on farm’s average milk yield
farm_mean_yield = train.groupby("Farm_ID")["Milk_Yield_L"].mean()
farm_rank = farm_mean_yield.rank(method="dense", ascending=False).astype(int)
train["Farm_ID"] = train["Farm_ID"].map(farm_rank)

# 4. Date → ordinal days since 2000-01-01
base_date = pd.to_datetime("2000-01-01")
train["Date_Ordinal"] = (train["Date"] - base_date).dt.days

train = train.drop(columns=["Date"])

In [109]:
train.to_csv("cleaned_train_data.csv", index=False)
print("Saved cleaned_train_data.csv")

Saved cleaned_train_data.csv


## Apply Same to Testing Set
Data Preparation: Cleaning

In [110]:
test = pd.read_csv(TEST_PATH)

# 1) REMOVE NEGATIVES WHERE THEY ARE INVALID
non_negative_cols = [
    "Age_Months", "Weight_kg", "Parity", "Days_in_Milk",
    "Feed_Quantity_kg", "Water_Intake_L", "Walking_Distance_km",
    "Grazing_Duration_hrs", "Resting_Hours", "Humidity_percent",
    "Previous_Week_Avg_Yield", "Milking_Interval_hrs", "Feed_Quantity_lb"
]

for col in non_negative_cols:
    if col in test.columns:
        test.loc[test[col] < 0, col] = np.nan

# 2) DROP RUMINATION_TIME_HRS (we removed it from train)
if "Rumination_Time_hrs" in test.columns:
    test = test.drop(columns=["Rumination_Time_hrs"])

# 3) IMPUTE HOUSING SCORE USING GROUP-BY FARM_ID + DATE (same as train)
test_hs = test.dropna(subset=["Housing_Score"])
farm_date_means = test_hs.groupby(["Farm_ID"])["Housing_Score"].mean()
farm_means = test_hs.groupby("Farm_ID")["Housing_Score"].mean()
global_mean_hs = train["Housing_Score"].mean()

def impute_hs(row):
    farm = row["Farm_ID"]
    if pd.isna(row["Housing_Score"]):
        # Try exact Farm + Date
        if farm in farm_date_means.index:
            return farm_date_means.loc[farm]
        # Fall back to Farm_ID mean
        if farm in farm_means.index:
            return farm_means.loc[farm]
        # Global fallback
        return global_mean_hs
    return row["Housing_Score"]

test["Housing_Score"] = test.apply(impute_hs, axis=1)


# 4) IMPUTE FEED_QUANTITY_KG USING BREED-GROUP MEANS (same as our final decision)
breed_means_fq = test.groupby("Breed")["Feed_Quantity_kg"].mean()
global_mean_fq = test["Feed_Quantity_kg"].mean()

def impute_feed(row):
    if pd.isna(row["Feed_Quantity_kg"]):
        b = row["Breed"]
        if b in breed_means_fq.index:
            return breed_means_fq.loc[b]
        return global_mean_fq
    return row["Feed_Quantity_kg"]

test["Feed_Quantity_kg"] = test.apply(impute_feed, axis=1)
test.drop(columns="Feed_Quantity_lb")
test.drop(columns="Cattle_ID")

# 6) DROP CLIMATE_ZONE (same as train)
test = test.drop("Climate_Zone", axis=1)

Feature Engineering:

In [111]:
# 1. One-hot encode same columns
test = pd.get_dummies(test, columns=onehot_cols, prefix=onehot_cols, drop_first=True)

# 2. Numeric encoding for Lactation_Stage
test["Lactation_Stage"] = test["Lactation_Stage"].map(stage_map).astype(int)

# 3. Farm_ID → use TRAIN-derived mapping
farmid_map = farm_rank.to_dict()
test["Farm_ID"] = test["Farm_ID"].map(farmid_map)

# Assign a fallback value for unseen farms
unknown_rank = max(farmid_map.values()) + 1
test["Farm_ID"] = test["Farm_ID"].fillna(unknown_rank).astype(int)

# 4. Date → ordinal days since 2000-01-01
test["Date"] = pd.to_datetime(test["Date"], errors="coerce")
test["Date_Ordinal"] = (test["Date"] - base_date).dt.days

# Drop the original Date column
test = test.drop(columns=["Date"])

# 5. Align one-hot encoded columns with train
train, test = train.align(test, join="left", axis=1, fill_value=0)

print("Test dataset fully cleaned + feature engineered.")

test.to_csv("cleaned_test_data.csv", index=False)
print("Saved cleaned_test_data.csv")

Test dataset fully cleaned + feature engineered.
Saved cleaned_test_data.csv


In [112]:
# lets run pca to reduce dimensions

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# 1. Separate features and target
target_col = "Milk_Yield_L"

feature_cols = [c for c in train.columns if c != target_col]

X_train = train[feature_cols]
y_train = train[target_col]

# For test we assume it has the same feature columns (and no milk_yield)
X_test = test[feature_cols]

# 2. Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Run PCA with adaptive components
pca_full = PCA().fit(X_train_scaled)

# choose minimum components needed for 95% variance explanation
target_variance = 0.95
cumulative = pca_full.explained_variance_ratio_.cumsum()
n_components = (cumulative < target_variance).sum() + 1

print("Chosen number of components:", n_components)

# fit final PCA with selected number
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# 4. Wrap into DataFrames
pc_cols = [f"PC{i+1}" for i in range(n_components)]

train_pca_df = pd.DataFrame(X_train_pca, columns=pc_cols, index=train.index)
train_pca_df[target_col] = y_train.values  # keep the target alongside PCs

test_pca_df = pd.DataFrame(X_test_pca, columns=pc_cols, index=test.index)

# 5. (Optional) PCA explained variance information
pca_info_df = pd.DataFrame({
    "PC": pc_cols,
    "explained_variance_ratio": pca.explained_variance_ratio_,
})
pca_info_df["cumulative_explained_variance"] = pca_info_df["explained_variance_ratio"].cumsum()

# 6. Save to CSV
train_pca_df.to_csv("train_pca.csv", index=False)
test_pca_df.to_csv("test_pca.csv", index=False)
pca_info_df.to_csv("pca_explained_variance.csv", index=False)

print("Saved train_pca.csv, test_pca.csv, and pca_explained_variance.csv")

Chosen number of components: 40
Saved train_pca.csv, test_pca.csv, and pca_explained_variance.csv
